In [1]:
import os
import requests
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.plot import show
from rasterio.transform import from_bounds
from shapely.geometry import box
import matplotlib.pyplot as plt

In [12]:
data_lookup = pd.read_csv("/media/becode/3D_House/Data/data_lookup.csv", sep="|")
data_lookup.DSM.apply(lambda x: "./Data/"x)
root = data_lookup.ROOT


In [6]:
entry = data_lookup[
        data_lookup.BOX.apply(lambda b:Box.from_string(b).in_bounds(165312, 194824))
       &data_lookup.PATH.apply(lambda p:p != "ROOT")]
entry

,ROOT,PATH,BOX,DSM,DTM,SHP
23615,k24,0_0_2_1_3,165000 194250 166000 194875 1000 625,./k24/0_0_2_1_3/DSM.tif,NaN,NaN


In [5]:
class Box:
    def __init__(self, bounds=None, left:int=0, bottom:int=0, right:int=0, top:int=0):
        if bounds is not None: self.left,self.bottom,self.right,self.top = bounds
        else: self.left,self.bottom,self.right,self.top = left,bottom,right,top
        self.width, self.height = self.right -self.left, self.top -self.bottom
    @classmethod
    def around_point(cls, x:int, y:int, size:int):
        return Box(None, x-size/2, y-size/2, x+size/2, y+size/2)
    @classmethod
    def from_string(cls, string:str=""):
        string = string.split(" ")
        return Box(None,
            int(string[0]),
            int(string[1]),
            int(string[2]),
            int(string[3]))
    def __repr__(self):
        return "{} {} {} {} {} {}".format(
            str(int(self.left)),
            str(int(self.bottom)),
            str(int(self.right)),
            str(int(self.top)),
            str(int(self.width)),
            str(int(self.height)))
    def __str__(self):
        return "left:{} bottom:{} right:{} top:{} width:{} height:{}".format(
            self.left, self.bottom, self.right, self.top, self.width, self.height)
    def in_bounds(self, x:int, y:int) -> bool:
        return self.left <= x < self.right and self.bottom <= y < self.top
    def box_in_bounds(self, other) -> bool:
        return other.left >= self.left and other.bottom >= self.bottom and other.right <= self.right and other.top <= self.top

In [7]:
def get_lambert(address:str) -> (int,int): 
    req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1")
    return (req.json()["LocationResult"][0]["Location"]["X_Lambert72"],
            req.json()["LocationResult"][0]["Location"]["Y_Lambert72"])

In [8]:
def get_containing_tif_old(x:int, y:int) -> rio.io.DatasetReader:
    return rio.open(data_lookup[
        data_lookup.BOX.apply(lambda b:Box.from_string(b).in_bounds(x,y))
       &data_lookup.PATH.apply(lambda p:p != "ROOT")].DSM.values[0])

In [9]:
def crop_location(tif, width:int=200, height:int=200) -> rio.io.DatasetReader:
    arr = np.array(tif.read(1))
    meta, bounds = tif.meta, tif.bounds
    posx, posy = int(x -bounds.left), int(abs(y -bounds.top))
    slicex = slice(posx -width//2, posx +width//2)
    slicey = slice(posy -height//2, posy +height//2)
    meta["width"], meta["height"] = width, height
    meta["transform"] = from_bounds(
        bounds.left + slicex.start,
        bounds.top - slicey.stop,
        bounds.left + slicex.stop,
        bounds.top - slicey.start,
        width, height)
    with rio.open("./crop.tif", "w", **meta) as crop:
        crop.write(arr[slicey,slicex], indexes=1)
    return rio.open("./crop.tif")

In [10]:
x, y = get_lambert("Sint-Pietersvliet 7, 2000 Antwerpen")
print(x,y)
tif = get_containing_tif_old(x, y)
plt.figure(figsize=(8,8))
show(crop_location(tif,100,100), cmap="cividis")

152242.43 212849.5


RasterioIOError: ./k15/1_2_1_1_0/DSM.tif: No such file or directory

In [253]:
def get_containing_tif(x:int, y:int, size:int=100) -> {rio.io.DatasetReader}:
    main = get_containing_tif_old(x,y)
    crop_box = Box.around_point(x,y,size)
    if Box(main.bounds).box_in_bounds(crop_box): return main
    result = {}
    result["main"] = get_containing_tif_old(x,y)
    box = Box(result["main"].bounds)
    crop_box = Box.around_point(x,y,size)
    if crop_box.left < box.left: result["left"]  = get_containing_tif_old(crop_box.left, y)
    if crop_box.bottom < box.bottom: result["bottom"]  = get_containing_tif_old(x, crop_box.bottom)
    if crop_box.right > box.right: result["right"]  = get_containing_tif_old(crop_box.right, y)
    if crop_box.top > box.top: result["top"]  = get_containing_tif_old(x, crop_box.top)        
    return result

x, y = get_lambert("Molenstraat 52, Sint-Kat")
tifs = get_containing_tif(x,y)

In [285]:
align = {
    "left": (1,0,1),
    "bottom": (0,1,0),
    "right": (0,1,1),
    "top": (1,0,0)}
def concat_tifs(tifs:(), key:str):
    meta = tifs[0].meta
    meta["width"] *= 2 if key is ()
    with open("./tmp.tif", "w", **meta)
    return np.concatenate((tifs[align[key][0]],tifs[align[key][1]]),axis=align[key][2])
concat_tifs((np.zeros(4).reshape(2,2), np.ones(4).reshape(2,2)),"bottom")

array([[0., 0.],
       [0., 0.],
       [1., 1.],
       [1., 1.]])

In [260]:
arr1 = np.arange(4).reshape(2,2)
arr2 = np.arange(4,8).reshape(2,2)
print(np.concatenate((arr2,arr1),axis=1)) #left
print(np.concatenate((arr1,arr2))) #bottom
print(np.concatenate((arr1,arr2),axis=1)) #right
print(np.concatenate((arr2,arr1))) #up

[[4 5 0 1]
 [6 7 2 3]]
[[0 1]
 [2 3]
 [4 5]
 [6 7]]
[[0 1 4 5]
 [2 3 6 7]]
[[4 5]
 [6 7]
 [0 1]
 [2 3]]


In [235]:
arr1 = np.arange(4).reshape(2,2)
arr2 = np.arange(4,8).reshape(2,2)
arr_LR = np.zeros(arr1.size +arr2.size).reshape(arr1.shape[0], arr1.shape[1] +arr2.shape[1])
arr_DU = np.zeros(arr1.size +arr2.size).reshape(arr1.shape[0] +arr2.shape[0], arr1.shape[1])
for y in range(arr_DU.shape[0]):
    for x in range(arr_DU.shape[1]):
        arr = 
for y in range(arr1.shape[0]):
    for x in range(arr1.shape[1]):
        arr3[y,x] = arr1[y,x]
    for x in range(arr2.shape[1]):
        arr3[y,x+arr1.shape[1]] = arr2[y,x]
print(arr3)
# A B right

[[0. 1. 4. 5.]
 [2. 3. 6. 7.]]


In [ ]:
arr1 = np.arange(4).reshape(2,2)
arr2 = np.arange(4,8).reshape(2,2)
arr3 = np.zeros(8).reshape() #TOP